In [29]:
## IMPORTS
import pandas as pd
import numpy as np
from datetime import date,datetime, timedelta
import yahoo_fin.stock_info as yf

In [30]:
# SectorDict = {
#     "Energy":"XLE",
#     "Technology":"XLK",
#     "Communication Services":"XLC",
#     "Consumer Defensive":"XLP",
#     "Healthcare":"XLV",
#     "Consumer Cyclical":"XLY",
#     "Industrials":"XLI",
#     "Utilities":"XLU",
#     "Basic Materials":"XLB",
#     "Financial Services":"XLF",
#     "Real Estate":"XLRE"}
# spList = yf.tickers_sp500()
# today = date.today()
# today = today.strftime('%Y-%m-%d')
dictionary = {}
# test = ['AAPL']
# for ticker in spList:
#     try:
#         # df = yf.get_data(ticker, start_date="1995-01-02",end_date="2022-01-09", index_as_date = True, interval="1wk")
#         # df.to_pickle('stockData/S&P500/'+ticker+'.pkl')
#         print(ticker)
#         industry = yf.get_company_info(ticker).loc['sector'][0]
#         dictionary[ticker] = SectorDict[industry]
#     except:
#         print("stock not found")

In [31]:
# print(dictionary)

In [32]:
import pickle
try:
    sectorDict = open('stockData/sector.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
  
except:
    print("Something went wrong")

In [33]:
import yahoo_fin.stock_info as yf
import numpy as np
import pandas as pd
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
now = datetime.now()
monday = now - timedelta(days = now.weekday())
monday = monday.strftime('%Y-%m-%d')
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(yf.get_data(sector, start_date="1998-12-21",end_date = monday, index_as_date = True, interval="1wk"))
    sectorDfList[-1].to_pickle("stockData/sectorCharts/"+sector+".pkl")

In [34]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spDf = yf.get_data('^GSPC', start_date="1998-12-21",end_date = monday, index_as_date = True, interval="1wk")
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iloc[index].close/df.iloc[index-30].close
def calculateSlope(df,index):
    if index < 31:
        return 0
    else:
        return df.iloc[index]['MA30']/df.iloc[index-1]['MA30']
for df in sectorDfList:
    df['MA30'] = df['close'].rolling(window=30).apply(lambda x: np.sum(x)/30)
    df['MA30Slope'] = df.apply(lambda x: calculateSlope(df,df.index.get_loc(x.name)),axis=1)
    df['percent'] = df.apply(lambda x: product(df,df.index.get_loc(x.name)),axis=1)
    spDf['percent'] = spDf.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
    df['RS'] = df['percent'] - spDf['percent']
    df.drop(['open','high','low','adjclose','volume','percent'],axis=1)
    df.to_pickle("stockData/sectorCharts/"+df.iloc[-1]['ticker']+".pkl")

In [35]:
goodSectorDf = sectorDfList[0].drop(['close','ticker','MA30','MA30Slope','RS'],axis=1)

In [36]:
from stage import fullPrint
goodSectorDf.index = sectorDfList[0].index
fullPrint(sectorDfList[1][['MA30Slope','RS']])

            MA30Slope        RS
1998-12-21   0.000000  0.000000
1998-12-28   0.000000  0.000000
1999-01-04   0.000000  0.000000
1999-01-11   0.000000  0.000000
1999-01-18   0.000000  0.000000
1999-01-25   0.000000  0.000000
1999-02-01   0.000000  0.000000
1999-02-08   0.000000  0.000000
1999-02-15   0.000000  0.000000
1999-02-22   0.000000  0.000000
1999-03-01   0.000000  0.000000
1999-03-08   0.000000  0.000000
1999-03-15   0.000000  0.000000
1999-03-22   0.000000  0.000000
1999-03-29   0.000000  0.000000
1999-04-05   0.000000  0.000000
1999-04-12   0.000000  0.000000
1999-04-19   0.000000  0.000000
1999-04-26   0.000000  0.000000
1999-05-03   0.000000  0.000000
1999-05-10   0.000000  0.000000
1999-05-17   0.000000  0.000000
1999-05-24   0.000000  0.000000
1999-05-31   0.000000  0.000000
1999-06-07   0.000000  0.000000
1999-06-14   0.000000  0.000000
1999-06-21   0.000000  0.000000
1999-06-28   0.000000  0.000000
1999-07-05   0.000000  0.000000
1999-07-12   0.000000  0.000000
1999-07-

In [37]:
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf['BadSectors'] = [[] for _ in range(len(goodSectorDf))]
for date in sectorDfList[0].index:
    listOfSector = []
    listOfBadSector = []
    for df in sectorDfList:
        try:
            if df.at[date,'MA30Slope'] > 1 and df.loc[date,'RS'] > 0.01:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    listOfSector.append(df.at[date,'ticker'])
            if (df.at[date,'MA30Slope'] < 0.995 and df.loc[date,'RS'] < 0) or df.at[date, 'MA30Slope'] < 0.975:
                if df.at[date,'close'] < df.at[date,'MA30']:
                    listOfBadSector.append(df.at[date,'ticker'])
        except: 
            print(df.iloc[-1]['ticker'])
    goodSectorDf.at[date,'Sectors'] = set(listOfSector)
    goodSectorDf.at[date,'BadSectors'] = set(listOfBadSector)

XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
X

In [38]:
from stage import fullPrint
fullPrint(goodSectorDf)
goodSectorDf.to_pickle('stockData/goodSector.pkl')

                  open        high        low   adjclose     volume   percent  \
1998-12-21   23.312500   23.812500  23.187500  13.875285      95300  0.000000   
1998-12-28   23.750000   23.750000  23.218750  13.710109      70300  0.000000   
1999-01-04   23.281250   24.562500  23.171875  14.077177     621400  0.000000   
1999-01-11   23.750000   23.781250  22.375000  13.343027     247300  0.000000   
1999-01-18   22.812500   22.906250  22.500000  13.269622      71800  0.000000   
1999-01-25   22.687500   22.687500  21.437500  12.810781     130400  0.000000   
1999-02-01   21.812500   22.937500  21.531250  13.434801     171000  0.000000   
1999-02-08   22.968750   23.218750  22.156250  13.012669     623800  0.000000   
1999-02-15   22.328125   22.437500  21.531250  12.755724     291600  0.000000   
1999-02-22   21.875000   22.093750  21.625000  12.700662     493100  0.000000   
1999-03-01   21.671875   23.031250  21.093750  13.425619     354800  0.000000   
1999-03-08   22.984375   25.